# EE315 Design Project

Create a Bingo card!

The below method generates an array of random numbers and sorts them into a bingo card format.

Importing the overlay class.

In [12]:
from pynq import Overlay

Program the Overlay onto the PYNQ-Z2, and parse the Hwh file

In [13]:
ol = Overlay("design_1_wrapper.bit")

Return a file that reports the results of the Hwh parsing to the designer.

In [14]:
ol?

Importing the required libraries.

In [15]:
import random
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Audio

In [16]:
rng = ol.rng_test01_0

In [17]:
rng.write(0x00, 0)
rng.write(0x04, 1)
rng.write(0x08,3)
out_address = 0x0C

In [18]:
my_list=[]
for i in range (1,91):
    my_list.append(i)

The call_number_func() ensures that the number generated cannot be greater than 90. If it is then it regenerates it and appends it to a list of random numbers. 

In [28]:
def call_number_func():

    RNG_number = rng.read(out_address)
    while RNG_number>len(my_list):
        RNG_number = rng.read(out_address)
        if RNG_number> len(my_list):
            RNG_number = 90 - len(my_list)
    
    TestNumber=22
    filename="./Sounds/"+str(TestNumber)+".mp3" #TestNumber should be RNG_number
    display(Audio(filename, autoplay =True))
    
    call_number = my_list[RNG_number]
    my_list.remove(call_number)

    call_list.append(call_number)

In [29]:
call_list=[]

In [30]:
call_number_func()

In [24]:
list(map(int, call_list))

[15]

The function implements the ability for the button to change colour. This will allow the user to interact and score off their numbers which have been called. 

In [12]:
button_list=[]
numPress=[]
wrongNumberList=[]

In [13]:
def button_on_click(myButton):
    myButton
    myButton.style.button_color = 'green'
    numPress.append(myButton.description)

In [14]:
#def bingo(validation):
#    if(validation):
#        print("Congratulations, you have won!")
#    else:
#        print("Unfortunately, your card is incorrect")     

In [15]:
#numMarked = []
#def numberCalledList(numPress):
    #for noCards in myCards:
        #n = 1
        #numMarked.append(numPress)
        #n = n + 1

In [25]:
def compareValues(numPress, call_list):
    wrongNumberList=[]
    list1=list(map(int, numPress)) #This changes list from strings to ints
    #list2=list(map(int, call_list))
    list2 = list(dict.fromkeys(call_list)) #converts to dict then back to list to remove duplicates
    check = all(item in list2 for item in list1)
    numReq = 2 #number of numbers required for bingo, 
    
    for element in list1:
        if element not in list2:
            wrongNumberList.append(element)
    print(list1)
    print("List 2 is ",list2)
    if len(list1)==numReq:
        if check is True: 
            print("Congratulations, it's Bingo!")
            display(Audio("./Sounds/ThatsABingo.mp3", autoplay =True))
        else:
            print("Unfortunately, these numbers do not match. These numbers are incorrect " + str(wrongNumberList))
    else:
        print("Unfortunately, there are not the right amount of numbers for Bingo")


In [26]:
numPress = [1,41]
call_list=[1,41]

In [27]:
compareValues(numPress, call_list)

[1, 41]
List 2 is  [1, 41]
Congratulations, it's Bingo!


In [19]:
def bingoPressed(bingoButton):
    bingoButton
    bingoButton.style.button_color = 'green'
    bingoButton = compareValues(numPress, call_list) #

Implementing the bingo card based upon the number of players and size of the card required. 

Through the use of a for loop, the numbers within the card can be generated and placed in a ipywidget layout. This layout creates a grid which is sized using the cardSize * cardSize. 

For the bingo game to be interactive, the need to implement a form of user interaction, such as scoring off the number called, was required. This is achieved by creating a for loop which has a callback element and calls the function above, button_on_click().

In [20]:
cardSize = 4
minNum = 1
maxNum = 90
noCards = 3
myCards = []
bingos = []

for i in range(noCards):
    card = []
    randRange = range(minNum, maxNum)
    card = random.sample(randRange, cardSize * cardSize)
 
    buttons = []
    
    myButton = widgets.GridBox(children=[widgets.Button(description=str(num), layout=widgets.Layout(width='auto', height='auto'),
                 button_style='warning') for num in card],

                layout=widgets.Layout(
                                    width='80%',
                                    grid_template_columns='20%'*cardSize,
                                    grid_template_rows='40px'*cardSize,
                                    grid_gap='1px 2px')
                           )
    
    for button in myButton.children:
        button.on_click(button_on_click)
            
    myCards.append(myButton)
    
  
    
    #bingoButton = widgets.GridBox(children=[widgets.Button(description='Bingo!', layout=widgets.Layout(width='auto', height='auto'),
                 #button_style='warning') for i in range(1)],

                #layout=widgets.Layout(
                                    #width='60%',
                                    #grid_template_columns='20%',
                                    #grid_template_rows='40px',
                                    #grid_gap='1px 2px')
                            
    
    bingoButton = widgets.Button(description='Bingo!', button_style='warning')
    
    #for bingo in bingoButton.children:
    bingoButton.on_click(bingoPressed)

    bingos.append(bingoButton)
    

The bingo card can then be displayed through calling myButton.

In [21]:
for noCards in myCards:
    numPress = []
    

In [22]:
bingo_buttons = []

In [31]:
numPress

['79', '6', '82']

In [32]:
#numPress = []
for idx, card in enumerate(myCards):
    #numPress = myCards[idx]
    display(card)
    display(bingos[idx])
#print(numPress)    

GridBox(children=(Button(button_style='warning', description='83', layout=Layout(height='auto', width='auto'),…

Button(button_style='warning', description='Bingo!', style=ButtonStyle(button_color='green'))

GridBox(children=(Button(button_style='warning', description='46', layout=Layout(height='auto', width='auto'),…

Button(button_style='warning', description='Bingo!', style=ButtonStyle(button_color='green'))

GridBox(children=(Button(button_style='warning', description='15', layout=Layout(height='auto', width='auto'),…

Button(button_style='warning', description='Bingo!', style=ButtonStyle())

Unfortunately, these numbers do not match. These numbers are incorrect []


In [52]:
bingos

[Button(button_style='warning', description='Bingo!', style=ButtonStyle()),
 Button(button_style='warning', description='Bingo!', style=ButtonStyle()),
 Button(button_style='warning', description='Bingo!', style=ButtonStyle())]

Prints a list of numbers - keep for now just incase. 

buttons = []
#for num in card:
    #print(num)
for num in card:
    print(num)
    myButton = widgets.Button(description=str(num), layout=widgets.Layout(width='100px', height='100px', columns='20%'*cardSize, rows='40px'*cardSize),
                         button_style='danger')

    buttons.append(myButton)
    myButton.on_click(button_on_click)

    
    

for button in buttons:
    display(button)

In [88]:
list(map(int, numPress))

[]

In [89]:
#CheckList=compare_lists(numPress,call_list)

Not enough numbers for Bingo


In [90]:
list(map(int, wrongNumberList))

[]